# 数据处理
## 1. 爬虫部分

### 1.1 计算爬取的总数据量（条）

In [ ]:
import pandas as pd

filepath = "DataCrawler/MyData/Data_.csv"
tot_len = 0
for i in range(1, 29):
    pathl = filepath[0:-4]
    pathr = filepath[-4:]
    nowpath = pathl + str(i) + pathr
    # print(nowpath)
    data = pd.read_csv(nowpath)
    tot_len += len(data)
    # print(data)
print(tot_len)

### 1.2 查看一个文件的数据

In [ ]:
filepath = "DataCrawler/MyData/Data_1.csv"
data = pd.read_csv(filepath)
print(data)

## 2. 打标签部分
### 2.1 将csv文件转成xlsx文件

In [ ]:
import pandas as pd
def csv_to_xlsx_pd(fileFrom, fileTo):
    csv = pd.read_csv('DataCrawler/MyData/' + fileFrom, encoding='utf-8')
    csv.to_excel('DataCrawler/MyXlsx/' + fileTo, sheet_name='data', index = False)

for i in range(1, 29):
    fileFrom = "Data_" + str(i) + ".csv"
    fileTo = "Data_" + str(i) + ".xlsx"
    csv_to_xlsx_pd(fileFrom, fileTo)

——**漫长的打标签**——

## 3. 添加成语的举例`example`以及解释`explanation`数据字段
### 3.1 重构json文件的数据

In [ ]:
import json

idiomDict = {}
# 读取idiom.json中的数据 重构
with open('idiom.json', 'r',encoding="utf-8") as f:
    lists = f.readlines()[0]
    preData = json.loads(lists)
    for data in preData:
        word = data["word"]
        idiomDict[word] = {}
        idiomDict[word]["explanation"] = data["explanation"]
        idiomDict[word]["example"] = data["example"]
# print(idiomDict) ############这里不要输出 否则JupyterNotebook会爆内存

### 3.2 添加字段并分割数据集

In [ ]:
import pandas as pd
import numpy as np  
import re
import random
from sklearn.utils import shuffle

# 保存为csv文件
def saveCsvData(datalist, savepath):
    if len(datalist) == 0:
        return
    data = pd.DataFrame(datalist)
    data.to_csv(savepath, index=False,
                header=['idiom1', 'idiom2', 'explanation1', 'example1', 'explanation2', 'example2', 'label'])

# 获取两个成语的解释和举例
def getExplanationAndExample(idiom1, idiom2):
    dic1 = idiomDict.get(idiom1)
    dic2 = idiomDict.get(idiom2)
    if dic1 == None or dic2 == None:               # 在idiomDict中查找 若没有 则跳过
        return (None, None, None, None)
    example1 = dic1['example']
    example2 = dic2['example']
    example1 = re.sub('(～)+', idiom_i[0], example1) # 使用正则表达式 将成语加入到造句中
    example2 = re.sub('(～)+', idiom_i[1], example2)
    example1 = re.sub('(★.*)+', "", example1)       # 使用正则表达式 去除造句来源
    example2 = re.sub('(★.*)+', "", example2)
    example1 = re.sub('(（.*)+', "", example1)
    example2 = re.sub('(（.*)+', "", example2)
    explanation1 = dic1['explanation']
    explanation2 = dic2['explanation']
    return (explanation1, example1, explanation2, example2)
    
allData = []

# 遍历所有Xlsx文件 从idiomDict中添加字段
for page in range(1, 2):
    file_path = "Data_" + str(page) + ".xlsx"
    data_xlsx = pd.read_excel(file_path)
    # print(data_xlsx)
    data_list = np.array(data_xlsx)

    for data_i in data_list:
        idiom_i = [data_i[0], data_i[1]]
        explanation1, example1, explanation2, example2 = getExplanationAndExample(data_i[0], data_i[1])
        if explanation1 == None:
            continue
        idiom_i.append(explanation1)
        idiom_i.append(example1)
        idiom_i.append(explanation2)
        idiom_i.append(example2)
        idiom_i.append(data_i[3])
        
        allData.append(idiom_i)

        
# 添加若干不具有逻辑关系的成语 与打标签数据个数相同
tot = len(allData)
for i in range(tot):
    idiomList = list(idiomDict.keys())
    idiomNum = len(idiomList)
    idiom1 = idiomList[random.randint(0,idiomNum)]
    idiom2 = idiomList[random.randint(0,idiomNum)]
    explanation1, example1, explanation2, example2 = getExplanationAndExample(idiom1, idiom2)
    idiom = [idiom1, idiom2, explanation1, example1, explanation2, example2, 0]
    allData.append(idiom)

allData = shuffle(allData)
# 拿出10%的数据用来测试
test_proportion = 0.1
test_idx = int(len(allData) * test_proportion)
# 分割训练集和测试集
train_data = allData[test_idx:]
test_data = allData[:test_idx]

savepath = "IdiomData_train.csv"
saveCsvData(train_data, savepath)
print(len(train_data))
savepath = "IdiomData_test.csv"
saveCsvData(test_data, savepath)
print(len(test_data))

* 读取单个CSV文件

In [ ]:
# 注意文件大的话 这里不要输出 否则JupyterNotebook会爆内存
filepath = "IdiomData_test.csv"
data = pd.read_csv(filepath)
print(data)

**至此，数据集构建完成**